In [1]:
import pickle

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import glob

pd.options.display.max_columns = None

In [2]:
#df_frag = pd.concat(map(pd.read_csv, glob.glob('data/csv_fragmentedV3/*.csv')))
#df = pd.concat(map(pd.read_csv, glob.glob('data/test/*.csv')))

In [3]:
df = pd.read_csv('data/cicids2017_kaggle/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')

In [6]:
df.shape

(286467, 79)

In [7]:
#df_cols = df.columns
#frag_cols = df_frag.columns

#common_cols = df_cols.intersection(frag_cols)
#only_frag_cols = frag_cols.difference(df_cols)

In [8]:
#df_frag = df_frag.drop(only_frag_cols, axis=1)

In [9]:
#df_frag['Label'] = df_frag['Label'].replace('No Label', 'Bot')

In [11]:
#df = pd.concat([df, df_frag], ignore_index=True, sort=False)

In [13]:
df = df.rename(columns={' Label': 'Label'})

In [14]:
le = LabelEncoder()
le.fit(df.Label)

LabelEncoder()

In [15]:
def reduce_anomalies(df, pct_anomalies=.01):
    labels = df['Label'].copy()
    is_anomaly = labels != 'BENIGN'
    num_normal = np.sum(~is_anomaly)
    num_anomalies = int(pct_anomalies * num_normal)
    all_anomalies = labels[labels != 'BENIGN']
    anomalies_to_keep = np.random.choice(
        all_anomalies.index, size=num_anomalies, replace=False)
    anomalous_data = df.iloc[anomalies_to_keep].copy()
    normal_data = df[~is_anomaly].copy()
    new_df = pd.concat([normal_data, anomalous_data], axis=0)
    return new_df

#reduce_anomalies(df)

In [16]:
labels = df['Label'].copy()


In [18]:
int_labels = le.transform(labels)


In [19]:
np.unique(int_labels, return_counts=True)

(array([0, 1]), array([127537, 158930]))

In [20]:
df = df.drop('Label', axis=1)

In [21]:
int_labels.shape

(286467,)

In [22]:
df.shape

(286467, 78)

In [23]:
# Remove infinities and NaNs
def remove_infs(df, labels):
    assert isinstance(df, pd.DataFrame)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep], labels[indices_to_keep]

In [24]:
before_removal = len(df)
df, int_labels = remove_infs(df, int_labels)
print(f'Length before NaN drop: {before_removal}, after NaN drop: {len(df)}\n \
The df is now {len(df)/before_removal} of its original size')

Length before NaN drop: 286467, after NaN drop: 286096
 The df is now 0.9987049119095742 of its original size


/tmp/ipykernel_971732/956947308.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)


In [25]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,286096.000000,2.860960e+05,286096.000000,286096.000000,286096.000000,2.860960e+05,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,286096.000000,286096.0,286096.0,286096.0,286096.000000,286096.000000,2.860960e+05,2.860960e+05,286096.000000,286096.000000,286096.000000,286096.000000,2.860960e+05,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.0,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.0,286096.0,286096.0,286096.0,286096.0,286096.0,286096.000000,286096.000000,286096.000000,2.860960e+05,286096.000000,286096.000000,286096.000000,286096.000000,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05
mean,8015.814957,5.386306e+06,3.476305,3.523953,233.703330,2.710754e+03,81.132277,10.360362,24.164366,24.622107,185.887101,27.510668,76.568685,55.814592,1.117593e+06,6.236320e+04,3.440116e+05,6.698980e+05,1.823650e+06,1.153528e+04,5.262043e+06,7.801051e+05,5.093240e+05,1.775746e+06,4.489433e+05,4.974906e+06,7.982842e+05,3.967066e+05,1.571719e+06,5.060200e+05,0.020968,0.0,0.0,0.0,91.943879,87.239150,4.146297e+04,2.113554e+04,10.135947,207.042874,47.919902,61.832647,3.561838e+04,0.008088,0.020968,0.000073,0.661110,0.123445,0.044719,0.0,0.000073,0.850617,55.221141,24.164366,76.568685,91.943879,0.0,0.0,0.0,0.0,0.0,0.0,3.476305,233.703330,3.523953,2.710754e+03,11150.420831,1146.227102,1.730720,29.080511,3.413634e+04,2.297963e+04,7.795819e+04,2.174195e+04,1.605051e+06,6.128327e+04,1.649324e+06,1.538278e+06
std,15346.254906,2.193700e+07,19.527596,28.307086,1866.714695,5.101078e+04,327.968919,24.252211,78.429008,117.290777,586.063169,54.237347,204.267870,196.760988,2.568310e+07,1.921187e+05,2.126341e+06,3.832840e+06,9.039912e+06,3.802998e+05,2.181016e+07,5.983210e+06,2.577312e+06,9.025005e+06,5.780141e+06,2.126673e+07,6.246715e+06,2.373165e+06,8.718060e+06,6.046578e+06,0.143279,0.0,0.0,0.0,503.881172,698.686067,1.602280e+05,6.249840e+04,18.778166,636.985751,122.100317,178.312136,1.632023e+05,0.089570,0.143279,0.008567,0.473333,0.328947,0.206687,0.0,0.008567,0.450324,129.866212,78.429008,204.267870,503.881172,0.0,0.0,0.0,0.0,0.0,0.0,19.527596,1866.714695,28.307086,5.101078e+04,14275.057102,6627.821754,14.906899,8.013593,4.662323e+05,2.624737e+05,7.347312e+05,4.225875e+05,8.687769e+06,1.217802e+06,8.888287e+06,8.566916e+06
min,0.000000,-1.300000e+01,1.000000,

In [26]:
# df = df.drop(['CWE Flag Count', 'Fwd Byts/b Avg','Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Bwd PSH Flags','Fwd URG Flags', 'Bwd URG Flags', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min'], axis=1)

In [27]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,286096.000000,2.860960e+05,286096.000000,286096.000000,286096.000000,2.860960e+05,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,286096.000000,286096.0,286096.0,286096.0,286096.000000,286096.000000,2.860960e+05,2.860960e+05,286096.000000,286096.000000,286096.000000,286096.000000,2.860960e+05,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.0,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.000000,286096.0,286096.0,286096.0,286096.0,286096.0,286096.0,286096.000000,286096.000000,286096.000000,2.860960e+05,286096.000000,286096.000000,286096.000000,286096.000000,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05,2.860960e+05
mean,8015.814957,5.386306e+06,3.476305,3.523953,233.703330,2.710754e+03,81.132277,10.360362,24.164366,24.622107,185.887101,27.510668,76.568685,55.814592,1.117593e+06,6.236320e+04,3.440116e+05,6.698980e+05,1.823650e+06,1.153528e+04,5.262043e+06,7.801051e+05,5.093240e+05,1.775746e+06,4.489433e+05,4.974906e+06,7.982842e+05,3.967066e+05,1.571719e+06,5.060200e+05,0.020968,0.0,0.0,0.0,91.943879,87.239150,4.146297e+04,2.113554e+04,10.135947,207.042874,47.919902,61.832647,3.561838e+04,0.008088,0.020968,0.000073,0.661110,0.123445,0.044719,0.0,0.000073,0.850617,55.221141,24.164366,76.568685,91.943879,0.0,0.0,0.0,0.0,0.0,0.0,3.476305,233.703330,3.523953,2.710754e+03,11150.420831,1146.227102,1.730720,29.080511,3.413634e+04,2.297963e+04,7.795819e+04,2.174195e+04,1.605051e+06,6.128327e+04,1.649324e+06,1.538278e+06
std,15346.254906,2.193700e+07,19.527596,28.307086,1866.714695,5.101078e+04,327.968919,24.252211,78.429008,117.290777,586.063169,54.237347,204.267870,196.760988,2.568310e+07,1.921187e+05,2.126341e+06,3.832840e+06,9.039912e+06,3.802998e+05,2.181016e+07,5.983210e+06,2.577312e+06,9.025005e+06,5.780141e+06,2.126673e+07,6.246715e+06,2.373165e+06,8.718060e+06,6.046578e+06,0.143279,0.0,0.0,0.0,503.881172,698.686067,1.602280e+05,6.249840e+04,18.778166,636.985751,122.100317,178.312136,1.632023e+05,0.089570,0.143279,0.008567,0.473333,0.328947,0.206687,0.0,0.008567,0.450324,129.866212,78.429008,204.267870,503.881172,0.0,0.0,0.0,0.0,0.0,0.0,19.527596,1866.714695,28.307086,5.101078e+04,14275.057102,6627.821754,14.906899,8.013593,4.662323e+05,2.624737e+05,7.347312e+05,4.225875e+05,8.687769e+06,1.217802e+06,8.888287e+06,8.566916e+06
min,0.000000,-1.300000e+01,1.000000,

In [28]:
# df = df[['Dst Port', 'Protocol']]

In [29]:
df.shape

(286096, 78)

In [33]:
x_train, x_test, y_train, y_test = train_test_split(df,
                                                    int_labels,
                                                    test_size=.15)

In [34]:
preprocessed_data = {
    'x_train': x_train,
    'y_train': y_train,
    'x_test': x_test,
    'y_test': y_test,
    'le': le
}

In [35]:
with open('data/preprocessed_data.pickle', 'wb') as file:
    pickle.dump(preprocessed_data, file)